# Organización de unidades productivas

## Limpieza de los datos
Eliminar filas innecesarias

In [88]:
import os

def clean_and_save_to_new_folder(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    archivos_csv = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['empadronada.']        
    headers = "Serie;Codigo_municipio;Nombre;2020;2021;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for archivo in archivos_csv:
        archivo_original = os.path.join(input_folder, archivo)
        archivo_limpiado = os.path.join(output_folder, archivo)
        with open(archivo_original, 'r') as f_input, open(archivo_limpiado, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = False
                    if 'Serie' in line:
                            delete = True
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-5:]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True # Eliminar filas posteriores

# Rutas de las carpetas de entrada y salida
input_folder = ".\original_data"
output_folder = ".\clean_data"

# Limpiar y guardar los archivos CSV en la nueva carpeta
clean_and_save_to_new_folder(input_folder, output_folder)


## Unión de datos por edad

In [89]:
import os
import csv
import re

def sumar_csvs(folder_path, output_path):
    # Crear un diccionario para almacenar los datos sumados
    datos_sumados = {}

    # Obtener la lista de archivos en la carpeta (ordenado por edad)
    archivos = sorted(os.listdir(folder_path), key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else float('inf'))
    # Iterar sobre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            with open(os.path.join(folder_path, archivo), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)

                # Obtener la extensión del nombre del archivo (por ejemplo, "0a4" o "5a9")
                intervalo_edad = archivo.split('_')[-1].split('.')[0]
                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in datos_sumados:
                        datos_sumados[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos de cada año para el intervalo de edad actual
                    for i, valor in enumerate(row[3:7], start=2020):
                        clave = f"{intervalo_edad}_{i}"
                        # Agregar los datos al diccionario enlazado
                        datos_sumados[(serie, codigo_municipio, nombre)][clave] = int(valor) if valor else 0

    # Escribir los datos sumados en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir los encabezados
        claves_totales = []

        for key, values in datos_sumados.items():
            if claves_totales == []:
                claves_totales.extend(values.keys())
        claves_totales = [str(clave) for clave in claves_totales]
        print(claves_totales)

        # Reemplazar '100' con 'mayores_100' si está presente en las claves totales
        claves_totales = [clave.replace('100', 'mayores_100') for clave in claves_totales]

        # Escribir los encabezados en el archivo CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *claves_totales])

            # Escribir los datos sumados
        for key, values in datos_sumados.items():
            # Crear una lista de valores para cada clave en el orden de las claves totales
            fila = [key[0], key[1], key[2]]
            for clave in claves_totales:
                fila.append(values.get(clave, 0))
            csv_writer.writerow(fila)



# Ruta de la carpeta que contiene los archivos CSV
folder_path = './clean_data'
# Ruta del archivo CSV de salida
output_path = './suma_por_edad/suma_por_edad.csv'
# Llamar a la función para procesar los archivos CSV
sumar_csvs(folder_path, output_path)

['0a4_2020', '0a4_2021', '0a4_2022', '0a4_2023', '5a9_2020', '5a9_2021', '5a9_2022', '5a9_2023', '10a14_2020', '10a14_2021', '10a14_2022', '10a14_2023', '15a19_2020', '15a19_2021', '15a19_2022', '15a19_2023', '20a24_2020', '20a24_2021', '20a24_2022', '20a24_2023', '25a29_2020', '25a29_2021', '25a29_2022', '25a29_2023', '30a34_2020', '30a34_2021', '30a34_2022', '30a34_2023', '35a39_2020', '35a39_2021', '35a39_2022', '35a39_2023', '40a44_2020', '40a44_2021', '40a44_2022', '40a44_2023', '45a49_2020', '45a49_2021', '45a49_2022', '45a49_2023', '50a54_2020', '50a54_2021', '50a54_2022', '50a54_2023', '55a59_2020', '55a59_2021', '55a59_2022', '55a59_2023', '60a64_2020', '60a64_2021', '60a64_2022', '60a64_2023', '65a69_2020', '65a69_2021', '65a69_2022', '65a69_2023', '70a74_2020', '70a74_2021', '70a74_2022', '70a74_2023', '75a79_2020', '75a79_2021', '75a79_2022', '75a79_2023', '80a84_2020', '80a84_2021', '80a84_2022', '80a84_2023', '85a89_2020', '85a89_2021', '85a89_2022', '85a89_2023', '90a94_